In [54]:
#! pip install PyAudio
#! pip install SpeechRecognition
import pandas as pd
from pycorenlp import StanfordCoreNLP
#!pip install azure-cognitiveservices-speech
#!pip install google-api-python-client 
#!pip install --upgrade google-cloud-speech
#!pip install anvil-uplink
#!pip install --upgrade google-api-python-client
#!pip install --upgrade google-cloud-speech

from azure.cognitiveservices.speech import AudioDataStream, SpeechConfig, SpeechSynthesizer, SpeechSynthesisOutputFormat
from azure.cognitiveservices.speech.audio import AudioOutputConfig
import pandas as pd
import numpy as np
import string
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk import pos_tag
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import anvil.server
anvil.server.connect("I6B7FZ3SMLFENTUGQOYGCTMC-NUKYRRNENRSFFQ4A")
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="C:/Users/HP/Downloads/core-phoenix-341914-55f3e91fad73.json"
import re
import nltk.corpus
from nltk.corpus import nps_chat
import pandas as pd
from sklearn.svm import LinearSVC
import nltk
import math
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('nps_chat')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package nps_chat to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package nps_chat is already up-to-date!


True

In [55]:
from sklearn.svm import LinearSVC
import collections
from nltk.metrics import precision
from nltk.metrics import recall
from nltk.metrics import ConfusionMatrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
class isQuestion():
    def __init__(self):
        posts = nltk.corpus.nps_chat.xml_posts()
        features = self.__get_feature_set(posts)
        new_features=[]
        for j in features: 
            label=j[1]
            if(label!='whQuestion' and label!='ynQuestion'):
                label='NotQuestion'
            new_features.append((j[0],label))
        features=new_features
        #self.classifier = self.naiveBayes(features)
        #self.classifier = self.DecisionTree(features)
        #self.classifier = self.MaxentClassifier(features)
        self.classifier = self.SVM(features)
        #self.classifier = self.KNN(features)
    def __get_feature_set(self, posts):
        feature = []
        for post in posts:
            post_text = post.text            
            features = {}
            words = nltk.word_tokenize(post_text)
            for word in words:
                features['contains({})'.format(word.lower())] = True
            feature.append((features, post.get('class')))
        return feature
    def naiveBayes(self, feature_set):
        training_size = int(len(feature_set) * 0.1)
        train_set, test_set = feature_set[training_size:], feature_set[:training_size]
        classifier = nltk.NaiveBayesClassifier.train(train_set)
        print("accuracy = ","{0:.5f}".format(nltk.classify.accuracy(classifier, test_set)))
        refsets = collections.defaultdict(set)
        testsets = collections.defaultdict(set)
        for i, (feats, label) in enumerate(test_set):
            refsets[label].add(i)
            observed = classifier.classify(feats)
            testsets[observed].add(i)
        print("\t\tWH-Question\t\tYN-Question\t\tNotQuestion\n")
        print( 'Precision:\t', "{0:.5f}".format(precision(refsets['whQuestion'], testsets['whQuestion'])), "\t\t", "{0:.5f}".format(precision(refsets['ynQuestion'], testsets['ynQuestion'])), "\t\t", "{0:.5f}".format(precision(refsets['NotQuestion'], testsets['NotQuestion'])))
        print( 'Recall:   \t', "{0:.5f}".format(recall(refsets['whQuestion'], testsets['whQuestion'])), "\t\t", "{0:.5f}".format(recall(refsets['ynQuestion'], testsets['ynQuestion'])), "\t\t", "{0:.5f}".format(recall(refsets['NotQuestion'], testsets['NotQuestion'])))
        return classifier
    def MaxentClassifier(self, feature_set):
        training_size = int(len(feature_set) * 0.1)
        train_set, test_set = feature_set[training_size:], feature_set[:training_size]
        classifier = nltk.MaxentClassifier.train(train_set,max_iter=15)
        print("accuracy = ","{0:.5f}".format(nltk.classify.accuracy(classifier, test_set)))
        refsets = collections.defaultdict(set)
        testsets = collections.defaultdict(set)

        for i, (feats, label) in enumerate(test_set):
            refsets[label].add(i)
            observed = classifier.classify(feats)
            testsets[observed].add(i)
        print("\t\tWH-Question\t\tYN-Question\t\tNotQuestion\n")
        print( 'Precision:\t', "{0:.5f}".format(precision(refsets['whQuestion'], testsets['whQuestion'])), "\t\t", "{0:.5f}".format(precision(refsets['ynQuestion'], testsets['ynQuestion'])), "\t\t", "{0:.5f}".format(precision(refsets['NotQuestion'], testsets['NotQuestion'])))
        print( 'Recall:   \t', "{0:.5f}".format(recall(refsets['whQuestion'], testsets['whQuestion'])), "\t\t", "{0:.5f}".format(recall(refsets['ynQuestion'], testsets['ynQuestion'])), "\t\t", "{0:.5f}".format(recall(refsets['NotQuestion'], testsets['NotQuestion'])))
        return classifier
    def DecisionTree(self, feature_set):
        training_size = int(len(feature_set) * 0.1)
        train_set, test_set = feature_set[training_size:], feature_set[:training_size]
        classifier = nltk.classify.SklearnClassifier(DecisionTreeClassifier(max_depth=200)).train(train_set)
       # print("accuracy = ","{0:.5f}".format(nltk.classify.accuracy(classifier, test_set)))
        refsets = collections.defaultdict(set)
        testsets = collections.defaultdict(set)
        for i, (feats, label) in enumerate(test_set):
            refsets[label].add(i)
            observed = classifier.classify(feats)
            testsets[observed].add(i)
#         print("\t\tWH-Question\t\tYN-Question\t\tNotQuestion\n")
#         print( 'Precision:\t', "{0:.5f}".format(precision(refsets['whQuestion'], testsets['whQuestion'])), "\t\t", "{0:.5f}".format(precision(refsets['ynQuestion'], testsets['ynQuestion'])), "\t\t", "{0:.5f}".format(precision(refsets['NotQuestion'], testsets['NotQuestion'])))
#         print( 'Recall:   \t', "{0:.5f}".format(recall(refsets['whQuestion'], testsets['whQuestion'])), "\t\t", "{0:.5f}".format(recall(refsets['ynQuestion'], testsets['ynQuestion'])), "\t\t", "{0:.5f}".format(recall(refsets['NotQuestion'], testsets['NotQuestion'])))
        return classifier
    def KNN(self, feature_set):
        training_size = int(len(feature_set) * 0.1)
        train_set, test_set = feature_set[training_size:], feature_set[:training_size]
        classifier = nltk.classify.SklearnClassifier(KNeighborsClassifier()).train(train_set)
        print("accuracy = ","{0:.5f}".format(nltk.classify.accuracy(classifier, test_set)))
        refsets = collections.defaultdict(set)
        testsets = collections.defaultdict(set)
        for i, (feats, label) in enumerate(test_set):
            refsets[label].add(i)
            observed = classifier.classify(feats)
            testsets[observed].add(i)
        print("\t\tWH-Question\t\tYN-Question\t\tNotQuestion\n")
        print( 'Precision:\t', "{0:.5f}".format(precision(refsets['whQuestion'], testsets['whQuestion'])), "\t\t", "{0:.5f}".format(precision(refsets['ynQuestion'], testsets['ynQuestion'])), "\t\t", "{0:.5f}".format(precision(refsets['NotQuestion'], testsets['NotQuestion'])))
        print( 'Recall:   \t', "{0:.5f}".format(recall(refsets['whQuestion'], testsets['whQuestion'])), "\t\t", "{0:.5f}".format(recall(refsets['ynQuestion'], testsets['ynQuestion'])), "\t\t", "{0:.5f}".format(recall(refsets['NotQuestion'], testsets['NotQuestion'])))
        return classifier
    def SVM(self, feature_set):
        training_size = int(len(feature_set) * 0.1)
        train_set, test_set = feature_set[training_size:], feature_set[:training_size]
        classifier = nltk.classify.SklearnClassifier(LinearSVC()).train(train_set)
        print("accuracy = ","{0:.5f}".format(nltk.classify.accuracy(classifier, test_set)))
        refsets = collections.defaultdict(set)
        testsets = collections.defaultdict(set)
        for i, (feats, label) in enumerate(test_set):
            refsets[label].add(i)
            observed = classifier.classify(feats)
            testsets[observed].add(i)
        print("\t\tWH-Question\t\tYN-Question\t\tNotQuestion\n")
        print( 'Precision:\t', "{0:.5f}".format(precision(refsets['whQuestion'], testsets['whQuestion'])), "\t\t", "{0:.5f}".format(precision(refsets['ynQuestion'], testsets['ynQuestion'])), "\t\t", "{0:.5f}".format(precision(refsets['NotQuestion'], testsets['NotQuestion'])))
        print( 'Recall:   \t', "{0:.5f}".format(recall(refsets['whQuestion'], testsets['whQuestion'])), "\t\t", "{0:.5f}".format(recall(refsets['ynQuestion'], testsets['ynQuestion'])), "\t\t", "{0:.5f}".format(recall(refsets['NotQuestion'], testsets['NotQuestion'])))
        return classifier
    def predict_question(self, text):
        words = nltk.word_tokenize(text.lower())        
        if '?' in text:
            return 1
        features = {}
        for word in words:
            features['contains({})'.format(word.lower())] = True            
        prediction_result = self.classifier.classify(features)
        if prediction_result == 'whQuestion' or prediction_result == 'ynQuestion':
            return 1
        return 0

In [56]:
def noOfSpeakers(speech_file):
    from google.cloud import speech_v1p1beta1 as speech
    client = speech.SpeechClient()
    
    gcs_uri=upload_to_bucket("meet_234",speech_file,"ml_jj1")
    
    with open(speech_file, "rb") as audio_file:
        content = audio_file.read()
        
    # The name of the audio file to transcribe
    gcs_uri = "gs://ml_jj1/meet_234"
    
    audio = speech.RecognitionAudio(uri=gcs_uri)
    diarization_config = speech.SpeakerDiarizationConfig(
      enable_speaker_diarization=True,
      min_speaker_count=2,
      max_speaker_count=10,
    )

    config = speech.RecognitionConfig(
        encoding = speech.RecognitionConfig.AudioEncoding.ENCODING_UNSPECIFIED,
        sample_rate_hertz=8000,
        language_code="en-US",
        diarization_config=diarization_config,
    )
    operation = client.long_running_recognize(config=config, audio=audio)
    response = operation.result(timeout=500)
    for result in response.results:
        words_info = result.alternatives[0].words
    speakers = []
    noOfSpeakers = 0
    for word_info in words_info:
        speakers.append(int(word_info.speaker_tag))
    speakers = set(speakers)
    delete_blob("meet_234","ml_jj1")
    return len(speakers)


In [57]:
def silience(file_name):
    from pydub import AudioSegment,silence
    myaudio = intro = AudioSegment.from_wav(file_name)
    silence = silence.detect_silence(myaudio, min_silence_len=1000, silence_thresh=-16)
    silence = [((start/1000),(stop/1000)) for start,stop in silence] #convert to sec
    return silence

In [58]:
#return the list of questions
def Question_detection(transcript):
    
    result = []
    obj = isQuestion()
    for i in transcript:
        if(obj.predict_question(i) == 1):
            result.append(i)
    return result
    #transcript is a list of setences ex. ["hello everyone","how is everyone","lets start todays class"]

In [59]:
def delete_blob(blob_name, bucket_name):
    """Deletes a blob from the bucket."""

    storage_client = storage.Client.from_service_account_json(
        'C:/Users/HP/Downloads/core-phoenix-341914-55f3e91fad73.json')

    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.delete()

In [60]:
from google.cloud import storage

def upload_to_bucket(blob_name, path_to_file, bucket_name):
    """ Upload data to a bucket"""

    # Explicitly use service account credentials by specifying the private key
    # file.
    storage_client = storage.Client.from_service_account_json(
        'C:/Users/HP/Downloads/core-phoenix-341914-55f3e91fad73.json')

    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.upload_from_filename(path_to_file)

    #returns a public url
    return blob.public_url

In [61]:
# Imports the Google Cloud client library
def speech_to_text(file_path):
    
    from google.cloud import speech

    gcs_uri=upload_to_bucket("meet_234",file_path,"ml_jj1")
    # Instantiates a client
    client = speech.SpeechClient()

    # The name of the audio file to transcribe
    gcs_uri = "gs://ml_jj1/meet_234"

    audio = speech.RecognitionAudio(uri=gcs_uri)

    config = speech.RecognitionConfig(
        #encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        encoding= speech.RecognitionConfig.AudioEncoding.ENCODING_UNSPECIFIED,
        sample_rate_hertz=16000,
        language_code="en-IN",enable_word_time_offsets=True,enable_automatic_punctuation=True,
    )

    # Detects speech in the audio file
    response = client.recognize(config=config, audio=audio)
    word_stamps=[]
    transcript=[]
    
    for result in response.results:
        alternative = result.alternatives[0]
        for word_info in alternative.words:
            word = word_info.word
            start_time = word_info.start_time
            end_time = word_info.end_time
            word_stamps.append([word,start_time,end_time])
        transcript.append(alternative.transcript)
    delete_blob("meet_234","ml_jj1")
    return [transcript,word_stamps]
   # return response.results 
#speech_to_text("test.mp3")

In [62]:
stopwords_list = stopwords.words('english')

lemmatizer = WordNetLemmatizer()

def my_tokenizer(doc):
    words = word_tokenize(doc)
    
    pos_tags = pos_tag(words)
    
    non_stopwords = [w for w in pos_tags if not w[0].lower() in stopwords_list]
    
    non_punctuation = [w for w in non_stopwords if not w[0] in string.punctuation]
    
    lemmas = []
    for w in non_punctuation:
        if w[1].startswith('J'):
            pos = wordnet.ADJ
        elif w[1].startswith('V'):
            pos = wordnet.VERB
        elif w[1].startswith('N'):
            pos = wordnet.NOUN
        elif w[1].startswith('R'):
            pos = wordnet.ADV
        else:
            pos = wordnet.NOUN
        
        lemmas.append(lemmatizer.lemmatize(w[0], pos))

    return lemmas

In [63]:
def ask_question(question,all_data,k,threshold,tfidf_vectorizer,tfidf_matrix):
    query_vect = tfidf_vectorizer.transform([question])
    similarity = cosine_similarity(query_vect, tfidf_matrix)[0]
    sim=[]
    for i in range(len(similarity)):
        sim.append((similarity[i],i))
    sim.sort(reverse=True)
    #use the top k results 
    interesting,difficult = 0,0
    for max_indices in range(k):
        max_index=sim[max_indices][1]
        max_sim = sim[max_indices][0]
        weight_diff = -1
        weight_interesting = -1
        if(all_data.iloc[max_index]['Is the question difficult?']=='Yes'): weight_diff=1
        if(all_data.iloc[max_index]['Is the question interesting?']=='Yes'): weight_interesting=1
        difficult+= weight_diff*max_sim
        interesting+= weight_interesting*max_sim
    
    result={}
    
    if(difficult>=threshold):
        result["difficulty"]="Yes"
    else:
        result["difficulty"]="No"
    
    if(interesting>=threshold):
        result["interesting"]="Yes"
    else:
        result["interesting"]="No"
        
    return result
    

In [64]:
def interesting_questions(Questions,df,tfidf_vectorizer,tfidf_matrix):
    diff,inter=0,0
    diffl,interl=[],[]
    for question in Questions:
        diffl.append(0)
        interl.append(0)
        response = ask_question(question,df,1,0,tfidf_vectorizer,tfidf_matrix)
        if(response["difficulty"]=="Yes"): 
            diff+=1
            diffl[-1]=1
        if(response["interesting"]=="Yes"): 
            inter+=1
            interl[-1]=1
    return [diff,inter,diffl,interl]

In [65]:
def init_data():
    #path to the form responses
    df = pd.read_csv("C:/Users/HP/Desktop/TARP_J/question_feedback.csv")
    df.iloc[0:,1:]
    df=df.rename(columns={'Write a question relevant to our subjects which profs ask(eg. What is NLP?, Can someone tell me the difference between ML and AI? etc.)':'Question'})
    
    tfidf_vectorizer = TfidfVectorizer(tokenizer=my_tokenizer)
    tfidf_matrix = tfidf_vectorizer.fit_transform(tuple(df['Question']))
    return [df,tfidf_matrix,tfidf_vectorizer]

In [66]:
def responseTime(transcript,word_stamps):
    obj = isQuestion()
    responseTimes=[]
    words=0
    total_time=0
    for i in range(len(transcript)):
        print(transcript[i],obj.predict_question(transcript[i]))
        try:
            if(obj.predict_question(transcript[i]) == 1):
                j=i            
                word_end_of_question=word_stamps[words+len(transcript[i].split(" "))-1][2]
                passed=words
                while(j<len(transcript) and obj.predict_question(transcript[j])==1):
                    passed+=len(transcript[j].split(" "))
                    j+=1
                if(j<len(transcript) and passed<len(word_stamps)):
                    if(passed+1<len(word_stamps)): passed+=1
                    word_start_of_response=word_stamps[passed+1][1]
                    responsetime = word_start_of_response-word_end_of_question
                    responseTimes.append(math.ceil(responsetime.total_seconds()))
            words+=len(transcript[i].split(" "))
        except:
            print(i)
    return responseTimes


In [67]:
def test_transcript(transcript):
    file1 = open('myfile.txt', 'w')
    file1.writelines(transcript)
    file1.close()

In [68]:
import anvil.media
from shutil import copy
transcript,word_stamps=[],[]
# @anvil.server.callable
# def write_media_to_file(media_object):
#     global transcript,word_stamps
#     with anvil.media.TempFile(media_object) as f:
#         #audio file stored in finally.mp3
#         copy(f, "finally.mp3")
#         #convert the speech to text
#         transcript,word_stamps=speech_to_text("finally.mp3")

In [73]:
@anvil.server.callable
def analyze():
    global transcript
    transcript,word_stamps=speech_to_text("finally.mp3")
    df,tfidf_matrix,tfidf_vectorizer=init_data()
    Questions = Question_detection(transcript)
    print(Questions)
    Response_times = responseTime(transcript,word_stamps)
    diff,inter,diffl,interl=interesting_questions(Questions,df,tfidf_vectorizer,tfidf_matrix)
    speaker_count=noOfSpeakers("finally.mp3")
    return [5,3,1,1,2,[2,1,3],[0,0,1],[0,1,0]]
    return [len(transcript),len(Questions),diff,inter,speaker_count,Response_times,diffl,interl]
#analyze()

accuracy =  0.94886
		WH-Question		YN-Question		NotQuestion

Precision:	 0.78689 		 0.68056 		 0.98050
Recall:   	 0.73846 		 0.76562 		 0.97627
[]
accuracy =  0.94886
		WH-Question		YN-Question		NotQuestion

Precision:	 0.78689 		 0.68056 		 0.98050
Recall:   	 0.73846 		 0.76562 		 0.97627
matches any sankshep releasing is it something never worship from the civil war and vivid dreams about reservation system having public never come across any other across the state itself another word for very long and CPK 0


In [74]:
@anvil.server.callable
def SpeechToText():
    global transcript
    transcript=["Hello, can you tell me what is machine learning?","machine learning is a branch of Artificial Intelligence and computer science which focuses on the use of data and algorithms to imitate the way humans learn","Can you tell me what are the different frameworks which are available in JavaScript?","the famous JavaScript frameworks are node.js view Angular JS and react.","Could you tell me the founders of Google?","the founders of Google are Larry Page and Sergey Brin"]
    return transcript

accuracy =  0.94886
		WH-Question		YN-Question		NotQuestion

Precision:	 0.78689 		 0.68056 		 0.98050
Recall:   	 0.73846 		 0.76562 		 0.97627
[]
accuracy =  0.94886
		WH-Question		YN-Question		NotQuestion

Precision:	 0.78689 		 0.68056 		 0.98050
Recall:   	 0.73846 		 0.76562 		 0.97627
matches any sankshep releasing is it something never worship from the civil war and vivid dreams about reservation system having public never come across any other across the state itself another word for very long and CPK 0
accuracy =  0.94886
		WH-Question		YN-Question		NotQuestion

Precision:	 0.78689 		 0.68056 		 0.98050
Recall:   	 0.73846 		 0.76562 		 0.97627
[]
accuracy =  0.94886
		WH-Question		YN-Question		NotQuestion

Precision:	 0.78689 		 0.68056 		 0.98050
Recall:   	 0.73846 		 0.76562 		 0.97627
matches any sankshep releasing is it something never worship from the civil war and vivid dreams about reservation system having public never come across any other across the state itself a

In [71]:
@anvil.server.callable
def updateTranscript(Transcript):
    global transcript
    newTranscript=[]
    for sentence in Transcript.split("\n"):
        newTranscript.append(sentence)
    transcript=newTranscript

accuracy =  0.94886
		WH-Question		YN-Question		NotQuestion

Precision:	 0.78689 		 0.68056 		 0.98050
Recall:   	 0.73846 		 0.76562 		 0.97627
[]
accuracy =  0.94886
		WH-Question		YN-Question		NotQuestion

Precision:	 0.78689 		 0.68056 		 0.98050
Recall:   	 0.73846 		 0.76562 		 0.97627
matches any sankshep releasing is it something never worship from the civil war and vivid dreams about reservation system having public never come across any other across the state itself another word for very long and CPK 0
accuracy =  0.94886
		WH-Question		YN-Question		NotQuestion

Precision:	 0.78689 		 0.68056 		 0.98050
Recall:   	 0.73846 		 0.76562 		 0.97627
[]
accuracy =  0.94886
		WH-Question		YN-Question		NotQuestion

Precision:	 0.78689 		 0.68056 		 0.98050
Recall:   	 0.73846 		 0.76562 		 0.97627
matches any sankshep releasing is it something never worship from the civil war and vivid dreams about reservation system having public never come across any other across the state itself a